This notebook show how I trained 5 EfficientnetB0 models using Autogluon to predict whether each image have these labels

(label 1 : Surveillance camera, label 2 : Car park, label 3 : Car, label 4 : Shrine, label 5 : Garden)

Note that if image have none of above label, image will be label as 0

### Preparing module and dataset

In [ ]:
!pip install --upgrade mxnet
!pip install autogluon

In [ ]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0

True
1


In [ ]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset
from autogluon.core.metrics import make_scorer

/usr/local/lib/python3.7/dist-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.0` and `torch==1.10.0+cu111` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/house-grade/
!mkdir /content/properties-tagging/
# !mkdir /content/thai-food-labeling/

!unzip /content/drive/MyDrive/super-ai-engineer-2021-house-grade-classification.zip -d /content/house-grade/
!unzip /content/drive/MyDrive/super-ai-engineer-2021-properties-tagging.zip -d /content/properties-tagging/
# !unzip /content/super-ai-engineer-2021-thai-food-labeling.zip -d /content/thai-food-labeling/

In [ ]:
!unzip /content/drive/MyDrive/archive_3.zip -d /content

In [ ]:
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm
import random
from sklearn.metrics import f1_score
import os

### Preparing dataset benchmark

Explore hackatron data benchmark

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/manual_label.csv')
train_df=train_df.reset_index()
train_df['image']=train_df['filename'].apply(lambda x: '/content/house-grade/train/'+x)
train_df.head()

,index,filename,labels,image
0,0,ChokChai4_img_13-7955799_100-6031141_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
1,1,ChokChai4_img_13-7956791_100-6031267_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
2,2,ChokChai4_img_13-7956791_100-6031267_a187-2159...,5,/content/house-grade/train/ChokChai4_img_13-79...
3,3,ChokChai4_img_13-7961753_100-6031881_a185-9785...,0,/content/house-grade/train/ChokChai4_img_13-79...
4,4,ChokChai4_img_13-7969811_100-5906061_a180-5812...,0,/content/house-grade/train/ChokChai4_img_13-79...


Explore teammate's augmented dataset benchmark

In [ ]:
augment_df=pd.read_csv('/content/House_price_data/data_augment_1_4.csv')
augment_df.head()

,filename,labels
0,House_price_data/augment/aug_label1_id18_angle...,1 3
1,House_price_data/augment/aug_label1_id18_angle...,1 3
2,House_price_data/augment/aug_label1_id18_angle...,1 3
3,House_price_data/augment/aug_label1_id18_angle...,1 3
4,House_price_data/augment/aug_label1_id18_angle...,1 3


Change augmented benchmark to be same format as hackatron benchmark

In [ ]:
augment_df['filename']=augment_df['filename'].str.split('/').str[-1]
augment_df

,filename,labels
0,aug_label1_id18_angle0_flip.jpg,1 3
1,aug_label1_id18_angle1_flip.jpg,1 3
2,aug_label1_id18_angle2_flip.jpg,1 3
3,aug_label1_id18_angle1.jpg,1 3
4,aug_label1_id18_angle2.jpg,1 3
...,...,...
1565,aug_label4_id5872_angle0_flip.jpg,4 5
1566,aug_label4_id5872_angle1_flip.jpg,4 5
1567,aug_label4_id5872_angle2_flip.jpg,4 5
1568,aug_label4_id5872_angle1.jpg,4 5


In [ ]:
augment_df['image']=augment_df['filename'].apply(lambda x: '/content/House_price_data/train/'+x)
augment_df.head()

,filename,labels,image
0,aug_label1_id18_angle0_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
1,aug_label1_id18_angle1_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
2,aug_label1_id18_angle2_flip.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
3,aug_label1_id18_angle1.jpg,1 3,/content/House_price_data/train/aug_label1_id1...
4,aug_label1_id18_angle2.jpg,1 3,/content/House_price_data/train/aug_label1_id1...


Concatenate hackatron benchmark with augmented benchmark

In [ ]:
new_train_df=pd.concat([train_df,augment_df],axis=0)
new_train_df

,index,filename,labels,image
0,0.0,ChokChai4_img_13-7955799_100-6031141_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
1,1.0,ChokChai4_img_13-7956791_100-6031267_a187-2159...,0,/content/house-grade/train/ChokChai4_img_13-79...
2,2.0,ChokChai4_img_13-7956791_100-6031267_a187-2159...,5,/content/house-grade/train/ChokChai4_img_13-79...
3,3.0,ChokChai4_img_13-7961753_100-6031881_a185-9785...,0,/content/house-grade/train/ChokChai4_img_13-79...
4,4.0,ChokChai4_img_13-7969811_100-5906061_a180-5812...,0,/content/house-grade/train/ChokChai4_img_13-79...
...,...,...,...,...
1565,NaN,aug_label4_id5872_angle0_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1566,NaN,aug_label4_id5872_angle1_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1567,NaN,aug_label4_id5872_angle2_flip.jpg,4 5,/content/House_price_data/train/aug_label4_id5...
1568,NaN,aug_label4_id5872_angle1.jpg,4 5,/content/House_price_data/train/aug_label4_id5...


In [ ]:
new_train_df=new_train_df.drop(['index','filename'],axis=1).set_index('image')
new_train_df

,labels
image,
/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg,5
/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg,0
/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg,0
...,...
/content/House_price_data/train/aug_label4_id5872_angle0_flip.jpg,4 5
/content/House_price_data/train/aug_label4_id5872_angle1_flip.jpg,4 5
/content/House_price_data/train/aug_label4_id5872_angle2_flip.jpg,4 5


In [ ]:
new_train_df.index

Index(['/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7970811_100-5906071_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7971811_100-5906081_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7972811_100-5906092_a180-5812875399312_s90_y0_f90_1.jpg',
       '/content/house-grade/train/ChokChai4_img_13-7973811_100-5906102_a180-5812875399312_s270_y0_f90_1.jpg',
    

Split single label column into multiple columns

In [ ]:
label1=[]
label2=[]
label3=[]
label4=[]
label5=[]

for labels in new_train_df['labels'].str.split(' '):

  if '1' in labels:
    label1.append(1)
  else:
    label1.append(0)

  if '2' in labels:
    label2.append(1)
  else:
    label2.append(0)

  if '3' in labels:
    label3.append(1)
  else:
    label3.append(0)

  if '4' in labels:
    label4.append(1)
  else:
    label4.append(0)

  if '5' in labels:
    label5.append(1)
  else:
    label5.append(0)

In [ ]:
new_train_df['label1']=label1
new_train_df['label2']=label2
new_train_df['label3']=label3
new_train_df['label4']=label4
new_train_df['label5']=label5

In [ ]:
new_train_df=new_train_df.drop('labels',axis=1)
new_train_df

,label1,label2,label3,label4,label5
image,,,,,
/content/house-grade/train/ChokChai4_img_13-7955799_100-6031141_a187-21598902886774_s270_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s270_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7956791_100-6031267_a187-21598902886774_s90_y0_f90_1.jpg,0,0,0,0,1
/content/house-grade/train/ChokChai4_img_13-7961753_100-6031881_a185-978546002526_s90_y0_f90_1.jpg,0,0,0,0,0
/content/house-grade/train/ChokChai4_img_13-7969811_100-5906061_a180-5812875399312_s270_y0_f90_1.jpg,0,0,0,0,0
...,...,...,...,...,...
/content/House_price_data/train/aug_label4_id5872_angle0_flip.jpg,0,0,0,1,1
/content/House_price_data/train/aug_label4_id5872_angle1_flip.jpg,0,0,0,1,1
/content/House_price_data/train/aug_label4_id5872_angle2_flip.jpg,0,0,0,1,1


Explore balance of benchmark after including augmented benchmark

In [ ]:
print(new_train_df['label1'].value_counts())
print(new_train_df['label2'].value_counts())
print(new_train_df['label3'].value_counts())
print(new_train_df['label4'].value_counts())
print(new_train_df['label5'].value_counts())

0    4100
1     423
Name: label1, dtype: int64
0    3354
1    1169
Name: label2, dtype: int64
0    2787
1    1736
Name: label3, dtype: int64
0    3259
1    1264
Name: label4, dtype: int64
0    3759
1     764
Name: label5, dtype: int64


In [ ]:
train_one=pd.DataFrame(new_train_df.loc[:,'label1']).rename(columns={'label1':'label'}).reset_index()
train_two=pd.DataFrame(new_train_df.loc[:,'label2']).rename(columns={'label2':'label'}).reset_index()
train_three=pd.DataFrame(new_train_df.loc[:,'label3']).rename(columns={'label3':'label'}).reset_index()
train_four=pd.DataFrame(new_train_df.loc[:,'label4']).rename(columns={'label4':'label'}).reset_index()
train_five=pd.DataFrame(new_train_df.loc[:,'label5']).rename(columns={'label5':'label'}).reset_index()

In [ ]:
train_three

,image,label
0,/content/house-grade/train/ChokChai4_img_13-79...,0
1,/content/house-grade/train/ChokChai4_img_13-79...,0
2,/content/house-grade/train/ChokChai4_img_13-79...,0
3,/content/house-grade/train/ChokChai4_img_13-79...,0
4,/content/house-grade/train/ChokChai4_img_13-79...,0
...,...,...
4518,/content/House_price_data/train/aug_label4_id5...,0
4519,/content/House_price_data/train/aug_label4_id5...,0
4520,/content/House_price_data/train/aug_label4_id5...,0
4521,/content/House_price_data/train/aug_label4_id5...,0


### Train model

Since hackatron's criteria is F1 score, we pick F1 score as metric

In [ ]:
ag_f1_scorer = make_scorer(name='f1_score',
                                 score_func=f1_score,
                                 optimum=1,
                                 greater_is_better=True)

Check all image model in Autogluon

In [ ]:
ImagePredictor.list_models()

('resnet18_v1',
 'resnet34_v1',
 'resnet50_v1',
 'resnet101_v1',
 'resnet152_v1',
 'resnet18_v2',
 'resnet34_v2',
 'resnet50_v2',
 'resnet101_v2',
 'resnet152_v2',
 'resnest14',
 'resnest26',
 'resnest50',
 'resnest101',
 'resnest200',
 'resnest269',
 'se_resnet18_v1',
 'se_resnet34_v1',
 'se_resnet50_v1',
 'se_resnet101_v1',
 'se_resnet152_v1',
 'se_resnet18_v2',
 'se_resnet34_v2',
 'se_resnet50_v2',
 'se_resnet101_v2',
 'se_resnet152_v2',
 'vgg11',
 'vgg13',
 'vgg16',
 'vgg19',
 'vgg11_bn',
 'vgg13_bn',
 'vgg16_bn',
 'vgg19_bn',
 'alexnet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'shufflenet_v1',
 'shufflenet_v2',
 'squeezenet1.0',
 'squeezenet1.1',
 'googlenet',
 'inceptionv3',
 'xception',
 'xception71',
 'mobilenet1.0',
 'mobilenet0.75',
 'mobilenet0.5',
 'mobilenet0.25',
 'mobilenetv2_1.0',
 'mobilenetv2_0.75',
 'mobilenetv2_0.5',
 'mobilenetv2_0.25',
 'mobilenetv3_large',
 'mobilenetv3_small',
 'cifar_resnet20_v1',
 'cifar_resnet56_v1',
 'cifar_resnet110

Train one EfficientnetB0 to predict label 1 (Surveillance camera)

In [ ]:
predictor_one = ImagePredictor(eval_metric=ag_f1_scorer)
predictor_one.fit(train_one, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Converting raw DataFrame to ImageDataset...
Detected 2 unique classes: [0, 1]
If you feel the `classes` is inaccurate, please construct the dataset explicitly, e.g. train_data = ImageDataset(train_data, classes=["foo", "bar"])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Randomly split train_data into train[4070]/validation[453] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != efficientnet_b0
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.train.epochs    200 != 20
root.train.early_stop_baseline 0.0 != -inf
root.misc.seed       42 != 631
}
Saved config to /content/2a29cfb4/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342d

Train another EfficientnetB0 to predict label 2 (Car park)

In [ ]:
predictor_two = ImagePredictor()
predictor_two.fit(train_two, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Train another EfficientnetB0 to predict label 3 (Car)

In [ ]:
predictor_three = ImagePredictor()
predictor_three.fit(train_three, hyperparameters={'epochs': 30, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Converting raw DataFrame to ImageDataset...
Detected 2 unique classes: [0, 1]
If you feel the `classes` is inaccurate, please construct the dataset explicitly, e.g. train_data = ImageDataset(train_data, classes=["foo", "bar"])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Randomly split train_data into train[4070]/validation[453] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != efficientnet_b0
root.misc.seed       42 != 513
root.train.early_stop_patience -1 != 10
root.train.epochs    200 != 30
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_max_value 1.0 != inf
}
Saved config to /content/b1fa151b/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342d

Train another EfficientnetB0 to predict label 4 (Shrine)

In [ ]:
predictor_four = ImagePredictor()
predictor_four.fit(train_four, hyperparameters={'epochs': 20, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Train another EfficientnetB0 to predict label 5 (Garden)

In [ ]:
predictor_five = ImagePredictor()
predictor_five.fit(train_five, hyperparameters={'epochs': 30, 'batch_size':32, 'model':'efficientnet_b0'},time_limit=60*60)

Save trained models

In [ ]:
predictor_one.save('/content/drive/MyDrive/autogluon_model/predictor1.ag')
predictor_two.save('/content/drive/MyDrive/autogluon_model/predictor2.ag')
predictor_three.save('/content/drive/MyDrive/autogluon_model/predictor3.ag')
predictor_four.save('/content/drive/MyDrive/autogluon_model/predictor4.ag')
predictor_five.save('/content/drive/MyDrive/autogluon_model/predictor5.ag')